In [17]:
import os
import matplotlib.pyplot as plt
import numpy as np
import torch
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
!pip install -q segmentation_models_pytorch
import segmentation_models_pytorch as smp
from tqdm.notebook import tqdm
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torch.utils.checkpoint as C
import torchvision.transforms.functional as fn
import torchvision.transforms as T
import matplotlib.pyplot as plt
!pip install -q torchsummary
from torchvision import models
from torchsummary import summary
import pandas as pd
from pathlib import Path
data_path = Path('/kaggle/input/google-research-identify-contrails-reduce-global-warming')

In [18]:
if torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')    
device

device(type='cuda')

In [19]:
_T11_BOUNDS = (243, 303)
_CLOUD_TOP_TDIFF_BOUNDS = (-4, 5)
_TDIFF_BOUNDS = (-4, 2)

def normalize_range(data, bounds):
    """Maps data to the range [0, 1]."""
    return (data - bounds[0]) / (bounds[1] - bounds[0])

In [ ]:
model = smp.UnetPlusPlus(encoder_name='resnet101',encoder_weights='imagenet',in_channels=3,classes=1,decoder_attention_type='scse',activation="sigmoid")
model.load_state_dict(torch.load('/kaggle/input/contrail-detection-run2/9th_epoch.pt'))
model.to(device)

In [21]:
train_path = '/kaggle/input/google-research-identify-contrails-reduce-global-warming/test'
ids=['1000834164244036115','1002653297254493116']

In [22]:
def rle_encode(x, fg_val=1):
    """
    Args:
        x:  numpy array of shape (height, width), 1 - mask, 0 - background
    Returns: run length encoding as list
    """

    dots = np.where(
        x.T.flatten() == fg_val)[0]  # .T sets Fortran order down-then-right
    run_lengths = []
    prev = -2
    for b in dots:
        if b > prev + 1:
            run_lengths.extend((b + 1, 0))
        run_lengths[-1] += 1
        prev = b
    return run_lengths


def list_to_string(x):
    """
    Converts list to a string representation
    Empty list returns '-'
    """
    if x: # non-empty list
        s = str(x).replace("[", "").replace("]", "").replace(",", "")
    else:
        s = '-'
    return s


def rle_decode(mask_rle, shape=(256, 256)):
    '''
    mask_rle: run-length as string formatted (start length)
              empty predictions need to be encoded with '-'
    shape: (height, width) of array to return 
    Returns numpy array, 1 - mask, 0 - background
    '''

    img = np.zeros(shape[0]*shape[1], dtype=np.uint8)
    if mask_rle != '-': 
        s = mask_rle.split()
        starts, lengths = [np.asarray(x, dtype=int) for x in (s[0:][::2], s[1:][::2])]
        starts -= 1
        ends = starts + lengths
        for lo, hi in zip(starts, ends):
            img[lo:hi] = 1
    return img.reshape(shape, order='F')  # Needed to align to RLE direction

In [23]:
import pandas as pd
from pathlib import Path
data_path = Path('/kaggle/input/google-research-identify-contrails-reduce-global-warming')
submission = pd.read_csv(data_path / 'sample_submission.csv', index_col='record_id')

for i in ids:
    band15= np.load(f'{train_path}/{i}/band_15.npy')
    band14= np.load(f'{train_path}/{i}/band_14.npy')
    band11= np.load(f'{train_path}/{i}/band_11.npy')
    r= normalize_range(band15 - band14, _TDIFF_BOUNDS)
    g= normalize_range(band14 - band11, _CLOUD_TOP_TDIFF_BOUNDS)
    b= normalize_range(band14, _T11_BOUNDS)
    image = np.clip((np.stack([r,g,b],axis=2)),0,1)
    image = image[...,4]
    image = torch.tensor(image).to(device)
    image = torch.moveaxis(image,-1,0)
    image = torch.unsqueeze(image,0)
    pred = model(image)
    pred = pred.detach().to('cpu')
    pred = pred.numpy()
    pred=np.squeeze(pred,(0,1))
    pred = np.c_[np.unravel_index(np.argpartition(pred.ravel(),-100)[-100:],pred.shape)]
    mask = np.zeros((256, 266))
    mask[pred[:, 0], pred[:, 1]] = 1
    # notice the we're converting rec to an `int` here:
    submission.loc[int(i), 'encoded_pixels'] = list_to_string(rle_encode(mask))

submission.head()

,encoded_pixels
record_id,
1000834164244036115,33 1 40 1 72 8 96 4 130 7 141 2 197 4 213 3 22...
1002653297254493116,1 1 63 6 90 4 98 8 113 6 123 12 142 2 187 1 20...


In [24]:
submission.to_csv('submission.csv')

model.eval()